In [1]:
import pandas as pd
import numpy as nm
import xgboost as xgb

In [2]:
import csv

In [3]:
xgboost_params = { 
   "objective": "binary:logistic",
   "booster": "gbtree",
   "eval_metric": "auc",
   "eta": 0.01, # 0.06, #0.01,
   #"min_child_weight": 240,
   "subsample": 0.75,
   "colsample_bytree": 0.68,
   "max_depth": 7
}

In [4]:
data = pd.read_csv("churn_data.csv")

In [5]:
data.head(5)

,Account Length,VMail Message,Day Mins,Eve Mins,Night Mins,Intl Mins,CustServ Calls,Churn,Int'l Plan,VMail Plan,Day Calls,Day Charge,Eve Calls,Eve Charge,Night Calls,Night Charge,Intl Calls,Intl Charge
0,128,25,265.1,197.4,244.7,10.0,1,0,0,1,110,45.07,99,16.78,91,11.01,3,2.70
1,107,26,161.6,195.5,254.4,13.7,1,0,0,1,123,27.47,103,16.62,103,11.45,3,3.70
2,137,0,243.4,121.2,162.6,12.2,0,0,0,0,114,41.38,110,10.30,104,7.32,5,3.29
3,84,0,299.4,61.9,196.9,6.6,2,0,1,0,71,50.90,88,5.26,89,8.86,7,1.78
4,75,0,166.7,148.3,186.9,10.1,3,0,1,0,113,28.34,122,12.61,121,8.41,3,2.73


In [6]:
data.shape

(3333, 18)

In [7]:
train = data[:2000]

In [8]:
train

,Account Length,VMail Message,Day Mins,Eve Mins,Night Mins,Intl Mins,CustServ Calls,Churn,Int'l Plan,VMail Plan,Day Calls,Day Charge,Eve Calls,Eve Charge,Night Calls,Night Charge,Intl Calls,Intl Charge
0,128,25,265.1,197.4,244.7,10.0,1,0,0,1,110,45.07,99,16.78,91,11.01,3,2.70
1,107,26,161.6,195.5,254.4,13.7,1,0,0,1,123,27.47,103,16.62,103,11.45,3,3.70
2,137,0,243.4,121.2,162.6,12.2,0,0,0,0,114,41.38,110,10.30,104,7.32,5,3.29
3,84,0,299.4,61.9,196.9,6.6,2,0,1,0,71,50.90,88,5.26,89,8.86,7,1.78
4,75,0,166.7,148.3,186.9,10.1,3,0,1,0,113,28.34,122,12.61,121,8.41,3,2.73
5,118,0,223.4,220.6,203.9,6.3,0,0,1,0,98,37.98,101,18.75,118,9.18,6,1.70
6,121,24,218.2,348.5,212.6,7.5,3,0,0,1,88,37.09,108,29.62,118,9.57,7,2.03
7,147,0,157.0,103.1,211.8,7.1,0,0,1,0,79,26.69,94,8.76,96,9.53,6,1.92
8,117,0,184.5,351.6,215.8,8.7,1,0,0,0,97,31.37,80,29.89,90,9.71,4,2.35
9,141,37,258.6,222.0,326.4,11.2,0,0,1,1,84,43.96,111,18.87,97,14.69,5,3.02


In [9]:
test = data[2000:]

In [10]:
test

,Account Length,VMail Message,Day Mins,Eve Mins,Night Mins,Intl Mins,CustServ Calls,Churn,Int'l Plan,VMail Plan,Day Calls,Day Charge,Eve Calls,Eve Charge,Night Calls,Night Charge,Intl Calls,Intl Charge
2000,28,0,168.2,161.7,192.4,10.1,3,0,0,0,87,28.59,92,13.74,112,8.66,3,2.73
2001,73,0,286.4,178.2,214.2,10.7,1,1,0,0,109,48.69,67,15.15,152,9.64,14,2.89
2002,156,0,174.3,186.6,258.2,12.9,3,0,0,0,95,29.63,128,15.86,105,11.62,5,3.48
2003,33,0,190.6,161.7,189.9,13.0,1,0,1,0,100,32.40,104,13.74,136,8.55,6,3.51
2004,77,0,175.5,205.1,245.2,17.8,4,0,0,0,86,29.84,78,17.43,100,11.03,3,4.81
2005,119,0,133.4,204.6,196.9,11.1,1,0,0,0,102,22.68,71,17.39,103,8.86,7,3.00
2006,91,27,204.6,136.0,210.5,6.6,3,0,0,1,96,34.78,93,11.56,82,9.47,2,1.78
2007,102,0,242.2,233.2,188.5,6.2,3,0,0,0,88,41.17,89,19.82,121,8.48,6,1.67
2008,86,33,253.1,210.1,95.0,11.9,3,0,0,1,112,43.03,94,17.86,98,4.27,4,3.21
2009,82,0,130.0,185.3,178.7,8.3,0,0,0,0,110,22.10,88,15.75,105,8.04,4,2.24


In [11]:
eval = test[:]

In [12]:
target = train["Churn"]

In [13]:
train = train.drop(["Churn"], axis=1)

In [14]:
test = test.drop(["Churn"], axis=1)

In [15]:
xgtrain = xgb.DMatrix(train.values, target.values)
xgtest = xgb.DMatrix(test.values)

In [16]:
boost_round = 1800
clf = xgb.train(xgboost_params,xgtrain,num_boost_round=boost_round,verbose_eval=True,maximize=False)

#Make predict
print('Predict...')
test_preds = clf.predict(xgtest, ntree_limit=clf.best_iteration)

Predict...


In [ ]:
results = pd.DataFrame(test_preds)

In [ ]:
results.to_csv("results.csv")

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
model = GradientBoostingClassifier(n_estimators=500, learning_rate=0.1, max_depth=3, random_state=0)
model.fit(train, target)
pred = model.predict(test)

In [ ]:
from sklearn.metrics import accuracy_score
ac = accuracy_score(eval['Churn'], pred)
print(ac)

In [ ]:
results_GBC = pd.DataFrame(pred)
results_GBC.to_csv("results_GBC.csv")